# 🧪 Spectral Affinity: Audio Restoration (The Boutique Lab)

This notebook implements a professional-grade restoration pipeline specifically tuned for AI-generated audio (e.g., Suno, Udio). 
It combines **Reference Matching** (to sound like your favorite tracks) with **Boutique Mastering** (to fix AI artifacts).

---

In [ ]:
!pip install -q pedalboard matchering numpy scipy torchaudio tqdm

In [ ]:
import os
import glob
import numpy as np
import scipy.signal as signal
import torch
import torchaudio
import matchering as mg
from tqdm.auto import tqdm
from pedalboard import Pedalboard, Compressor, Distortion, Gain, HighpassFilter, LowpassFilter, HighShelfFilter, Limiter
from pedalboard.io import AudioFile
from IPython.display import FileLink

# --- SETTINGS ---
INPUT_DIR = "/kaggle/input/datasets/danieldobles/ost-songs-a"
REF_FILE = "/kaggle/input/datasets/danieldobles/ost-songs-a/REF.flac"
OUTPUT_DIR = "/kaggle/working/mastered_tracks"
# ----------------

def mono_maker(audio_side, sample_rate, cutoff_hz=120):
    sos = signal.butter(4, cutoff_hz, 'hp', fs=sample_rate, output='sos')
    processed_side = signal.sosfilt(sos, audio_side)
    return processed_side

def ms_encode(audio_lr):
    mid = (audio_lr[0] + audio_lr[1]) * 0.5
    side = (audio_lr[0] - audio_lr[1]) * 0.5
    return mid, side

def ms_decode(mid, side):
    left = mid + side
    right = mid - side
    return np.stack([left, right])

def spectral_deharsh(audio_array, sample_rate, threshold_ratio=1.4):
    f, t, Zxx = signal.stft(audio_array, fs=sample_rate, nperseg=2048)
    mag = np.abs(Zxx)
    envelope = signal.medfilt2d(mag, kernel_size=(31, 1))
    mask = mag > (envelope * threshold_ratio)
    gain_map = np.ones_like(mag)
    reduction = np.clip(envelope[mask] / (mag[mask] + 1e-6), 0.5, 1.0)
    gain_map[mask] = reduction
    Zxx_clean = Zxx * gain_map
    _, audio_clean = signal.istft(Zxx_clean, fs=sample_rate)
    return audio_clean[:len(audio_array)] if len(audio_clean) > len(audio_array) else np.pad(audio_clean, (0, len(audio_array) - len(audio_clean)))

def transient_shaper_mid(mid_signal, sample_rate, punch=1.4):
    abs_sig = np.abs(mid_signal)
    sos_fast = signal.butter(1, 40, 'low', fs=sample_rate, output='sos')
    sos_slow = signal.butter(1, 5, 'low', fs=sample_rate, output='sos')
    env_fast = signal.sosfiltfilt(sos_fast, abs_sig)
    env_slow = signal.sosfiltfilt(sos_slow, abs_sig)
    transient_ratio = env_fast / (env_slow + 1e-8)
    gain_curve = np.where(transient_ratio > 1.05, transient_ratio ** (punch - 1.0), 1.0)
    return mid_signal * np.clip(gain_curve, 1.0, 2.0)

def saturate_side(side_signal, sample_rate, drive=4.0):
    side_expanded = side_signal[None, :]
    board = Pedalboard([HighpassFilter(300), Distortion(drive_db=drive), Gain(-1)])
    return board(side_expanded, sample_rate).squeeze()

def boutique_master(audio_lr, sample_rate):
    mid, side = ms_encode(audio_lr)
    side = mono_maker(side, sample_rate, 120)
    mid = transient_shaper_mid(mid, sample_rate, 1.4)
    side = saturate_side(side, sample_rate, 4.0)
    stereo = ms_decode(mid, side)
    clean_l = spectral_deharsh(stereo[0], sample_rate)
    clean_r = spectral_deharsh(stereo[1], sample_rate)
    final = Pedalboard([Limiter(threshold_db=-1.0)])(np.stack([clean_l, clean_r]), sample_rate)
    return final

print("✅ Boutique Audio Lab 2.0 Ready.")

### 🧪 Execution Pipeline: Matching + Restoration

In [ ]:
os.makedirs(OUTPUT_DIR, exist_ok=True)
temp_match_dir = "/kaggle/working/temp_matched"
os.makedirs(temp_match_dir, exist_ok=True)

file_paths = glob.glob(os.path.join(INPUT_DIR, "*.mp3")) + glob.glob(os.path.join(INPUT_DIR, "*.wav"))
print(f"🎵 Starting Pipeline for {len(file_paths)} tracks using Reference: {os.path.basename(REF_FILE)}")

for path in tqdm(file_paths, desc="🔬 Processing"):
    try:
        fname = os.path.basename(path)
        temp_matched = os.path.join(temp_match_dir, f"matched_{fname}.wav")
        final_output = os.path.join(OUTPUT_DIR, f"Mastered_{fname}.wav")
        
        # Step 1: Matching with Reference (EQ/RMS/Width)
        # The argument is 'result' or 'results' depending on version, usually 'result' in Matchering 2.0+
        mg.process(target=path, reference=REF_FILE, result=temp_matched)
        
        # Step 2: Boutique Polish (Transient shaping + De-harshing)
        with AudioFile(temp_matched) as f:
            audio = f.read(f.frames)
            sr = f.samplerate
            
        mastered_audio = boutique_master(audio, sr)
        
        with AudioFile(final_output, 'w', sr, mastered_audio.shape[0]) as f:
            f.write(mastered_audio)
            
    except Exception as e:
        print(f"❌ Failed {fname}: {str(e)}")

print(f"\n✨ All tracks processed successfully.")

### 📦 Final Download

In [ ]:
!zip -0 -rq /kaggle/working/restoration_results.zip mastered_tracks
print("✅ ZIP Bundle generated successfully.")
display(FileLink('/kaggle/working/restoration_results.zip'))